In [4]:
import sys
print(sys.platform)

import platform
platform.system()


linux


'Linux'

# Setup

In [ ]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
%pwd

/kaggle/working


In [ ]:
!pip install -q --upgrade uv
!uv sync

Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 1 package in 2ms                                            
Audited in 0.01ms


# Running

In [ ]:
!pwd

/kaggle/working/mlproduction
